In [214]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from graphviz import Source
from sklearn import metrics
from sklearn import tree, ensemble
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    confusion_matrix,
    roc_curve,
    auc)
from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.decomposition import PCA

In [215]:
train = pd.read_csv("train.csv")
pd.set_option('display.max_columns', None)

#remove the Id column as we will not use it
train = train.iloc[:,1:]
train

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,2607,243,23,258,7,660,170,251,214,1282,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
15116,2603,121,19,633,195,618,249,221,91,1325,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
15117,2492,134,25,365,117,335,250,220,83,1187,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
15118,2487,167,28,218,101,242,229,237,119,932,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [216]:
x = train.iloc[:, :-1]
y = train.loc[:, "Cover_Type"]

In [217]:
#Create function to find and replace outliers from outlier_features

def find_replace_outliers(data):
    
    outlier_features = ['Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points']
    
    for feature in outlier_features:

        q75,q25 = np.percentile(data[feature],[75,25])
        intr_qr = q75-q25

        maximum = q75+(1.5*intr_qr)
        minimum = q25-(1.5*intr_qr)

        data[feature] = np.where(
            (data[feature] > maximum), maximum, np.where(
            (data[feature] < minimum), minimum, data[feature]))

In [218]:
find_replace_outliers(x)

C:\Users\ivanh\AppData\Local\Temp/ipykernel_43900/2336048662.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feature] = np.where(


In [219]:
def feature_engineering(data):
    
    #Create feature 'Direct_Distance_To_Hydrology' by calculating hypothenus using 'Horizontal_Distance_To_Hydrology' and 'Vertical_Distance_To_Hydrology'
    
    data['Direct_Distance_To_Hydrology'] = np.where(
            (data['Horizontal_Distance_To_Hydrology'] == 0),
            data['Vertical_Distance_To_Hydrology'],
            np.where(
                (data['Vertical_Distance_To_Hydrology'] == 0),
                data['Horizontal_Distance_To_Hydrology'],
                (((data['Horizontal_Distance_To_Hydrology']**2)+(data['Vertical_Distance_To_Hydrology']**2))**0.5)*(data['Vertical_Distance_To_Hydrology']/abs(data['Vertical_Distance_To_Hydrology']))))
    
    #Create feature 'Rockness' using the information provided in Kaggle to categorize Soil Rockness as Undefined_Complex, Rubbly, Stony, Very_Stony or Extremely_Stony_Bouldery
    
    data['Rockness'] = 0

    Undefined_Complex = [7, 15, 8, 14, 16, 17, 19, 20, 21, 23]
    for num in Undefined_Complex:
        a = num
        data['Rockness'] = np.where(data[f'Soil_Type{a}']==1, 'Undefined_Complex', data['Rockness'])

    Rubbly = [3, 4, 5, 10, 11, 13]
    for num in Rubbly:
        a = num
        data['Rockness'] = np.where(data[f'Soil_Type{a}']==1, 'Rubbly', data['Rockness'])

    Stony = [6, 12]
    for num in Stony:
        a = num
        data['Rockness'] = np.where(data[f'Soil_Type{a}']==1, 'Stony', data['Rockness'])

    Very_Stony = [2, 9, 18, 26]
    for num in Very_Stony:
        a = num
        data['Rockness'] = np.where(data[f'Soil_Type{a}']==1, 'Very_Stony', data['Rockness'])

    Extremely_Stony_Bouldery = [1, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 22, 35]
    for num in Extremely_Stony_Bouldery:
        a = num
        data['Rockness'] = np.where(data[f'Soil_Type{a}']==1, 'Extremely_Stony_Bouldery', data['Rockness'])
        
    #Setting 'Rockness' as a ordinal categorical variable
    
    data['Rockness'] = np.where(data['Rockness'] == 'Undefined_Complex', 1, np.where(
    data['Rockness'] == 'Rubbly', 2, np.where(
        data['Rockness'] == 'Stony', 3, np.where(
            data['Rockness'] == 'Very_Stony', 4, 5))))
    
    #Create 'Avg_Hillshade_9_To_3' in order to explain Hillshade throughout the day in one single variable
    
    data['Avg_Hillshade_9_To_3'] = (data['Hillshade_9am']+data['Hillshade_Noon']+data['Hillshade_3pm'])/3
    
    #Reverse Onehot Encoding for 'Wilderness_Area' and 'Soil_Type'
    
    wilderness_cols = []
    for col in data.columns:
        if col in data.filter(regex='Wilderness').columns:
            wilderness_cols.append(col)
    
    soil_cols = []
    for col in data.columns:
        if col in data.filter(regex='Soil_Type').columns:
            soil_cols.append(col)
            
    data['Wilderness'] = data[wilderness_cols].idxmax(1)
    data['Soil'] = data[soil_cols].idxmax(1)
    
    cols_to_drop = data.filter(regex='Wilderness_Area|Soil_Type').columns
    data.drop(cols_to_drop, axis=1, inplace=True)
    
    #Set up correct data types for columns
    
    for col in data.columns:
        if col in data.filter(regex='Wilderness|Soil|Rockness').columns:
            data[col] = data[col].astype('category')

In [220]:
feature_engineering(x)

In [221]:
cols_to_drop = pd.DataFrame(train.filter(regex='Soil_Type').columns)

In [222]:
cols_to_drop['Soil'] = cols_to_drop[0]

In [223]:
test = pd.read_csv("test.csv")
test = test.iloc[:,1:]
feature_engineering(test)

In [224]:
categorical_features = list(x.select_dtypes(include=["category"]).columns)
numerical_features = list(set(x.columns) - set(categorical_features))

In [225]:
categorical_features

['Rockness', 'Wilderness', 'Soil']

In [226]:
categorical_features.remove('Soil')
categorical_features2 = ['Soil']

In [227]:
ord_encoder_transformer = ColumnTransformer(
    [
        (
            "ordinal_encoder", 
            OrdinalEncoder(), 
            categorical_features
        )
    ],
        remainder="passthrough"
)

In [228]:
x_encoded = pd.DataFrame(
    ord_encoder_transformer.fit_transform(x[categorical_features]),
    columns=categorical_features
)

In [229]:
soil_encoder_transformer = ColumnTransformer(
    [
        (
            "ordinal_encoder", 
            OrdinalEncoder(), 
            categorical_features2
        )
    ],
        remainder="passthrough"
)

In [230]:
soil_encoder_transformer.fit(cols_to_drop[categorical_features2])

ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinal_encoder', OrdinalEncoder(),
                                 ['Soil'])])

In [231]:
x_soil_encoded = pd.DataFrame(
    soil_encoder_transformer.transform(x[categorical_features2]),
    columns=categorical_features2
)

In [232]:
standard_scaler = ColumnTransformer(
    [
        (
            "robust_scaler", 
            StandardScaler(), 
            numerical_features
        )
    ],
        remainder="passthrough"
)

In [233]:
x_scaled = pd.DataFrame(
    standard_scaler.fit_transform(x[numerical_features]),
    columns=numerical_features
)

In [234]:
test_encoded = pd.DataFrame(ord_encoder_transformer.transform(test[categorical_features]),columns=categorical_features)
test_soil_encoded = pd.DataFrame(soil_encoder_transformer.transform(test[categorical_features2]),columns=categorical_features2)
test_scaled = pd.DataFrame(standard_scaler.transform(test[numerical_features]), columns=numerical_features)

In [235]:
x_scaled = pd.concat([x_scaled, x_soil_encoded, x_encoded], axis=1)
test_scaled = pd.concat([test_scaled, test_soil_encoded, test_encoded], axis=1)

In [236]:
x_scaled = x_scaled.drop(['Avg_Hillshade_9_To_3'], axis=1)
test_scaled = test_scaled.drop(['Avg_Hillshade_9_To_3'], axis=1)

In [238]:
tree_model = ExtraTreesClassifier(random_state = 17, 
                                 n_estimators = 135,
                                  max_features = 0.7,
                                  max_depth = 35, 
                                  min_samples_leaf = 1)

tree_model.fit(x_scaled, y)
Submition = pd.DataFrame(tree_model.predict(test_scaled))
Submition

,0
0,5
1,1
2,1
3,2
4,5
...,...
565887,3
565888,3
565889,3
565890,3


In [239]:
Submition.to_csv("submition.csv")